# Dynamic Pricing for Urban Parking Lots
**Capstone Project – Summer Analytics 2025**

In [ ]:
#  Install required packages
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Imports
import numpy as np
import pandas as pd
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

In [ ]:
# Load your dataset (upload manually via Colab file upload UI)
df = pd.read_csv("/content/dataset.csv")

# Merge date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Save minimal streamable format
df[['Timestamp', 'Occupancy', 'Capacity']].to_csv("parking_stream.csv", index=False)

In [ ]:
# Define Pathway schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

# Load data as simulated real-time stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
#  Model 1: Baseline Linear Pricing
base_price = 10
alpha = 5

baseline_model = data_with_time.with_columns(
    price = base_price + alpha * (data_with_time.Occupancy / data_with_time.Capacity)
)

In [ ]:
#  Model 2: Demand-Based Pricing
data_with_features = data_with_time.with_columns(
    queue_length = data_with_time.Occupancy % 10,
    traffic = data_with_time.Occupancy % 5,
    is_special_day = pw.if_else(data_with_time.Occupancy % 7 == 0, 1, 0),
    vehicle_type_weight = (data_with_time.Occupancy % 3) * 0.1
)

data_with_demand = data_with_features.with_columns(
    demand = 0.6 * (data_with_features.Occupancy / data_with_features.Capacity)
           + 0.2 * data_with_features.queue_length
           - 0.1 * data_with_features.traffic
           + 0.1 * data_with_features.is_special_day
           + 0.05 * data_with_features.vehicle_type_weight,
)

max_demand = 5.0

data_with_demand = data_with_demand.with_columns(
    norm_demand = data_with_demand.demand / max_demand,
)

data_with_demand = data_with_demand.with_columns(
    price = pw.apply(lambda d: min(max(base_price * (1 + 0.5 * d), 5), 20), data_with_demand.norm_demand)
)

In [18]:
# Visualization with Bokeh
def plot_price(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Real-Time Parking Price (Model 2)",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

viz = data_with_demand.plot(plot_price, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [19]:
# Run the Pathway pipeline
%%capture --no-display
pw.run()

Output()